In [1]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import urllib.request as req
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options

import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

book_cate = {'소설':'01','시/에세이':'03','인문':'05','경제/경영':'13','자기계발':'15'}
# 01 : 소설 , 03 : 시/에세이 , 05 : 인문 , 13 : 경제/경영 , 15 : 자기계발
url = 'https://product.kyobobook.co.kr/category/KOR/'+book_cate['인문']+'#?page=1&type=best&per=20'

User_Agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
chrome_options = Options()
chrome_options.add_argument(f"user-agent={User_Agent }")

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

book_titles = list()
book_stories = list()
book_keyPicks = list()
keywordPick = list()
book_page_urls = list()


for i in range(1,51):
    page_links = driver.find_elements(By.CSS_SELECTOR,'#homeTabBest > div.switch_prod_wrap.view_type_list > ol > li > div.prod_area.horizontal > div.prod_info_box > a')
    for page_link in page_links:
        link = page_link.get_attribute('href')
        book_page_urls.append(link)
    if i== 50:
       break
    next_page = driver.find_element(By.CSS_SELECTOR,'#bestBottomPagi > button.btn_page.next')
    next_page.send_keys(Keys.ENTER)
    time.sleep(3)
driver.quit()


In [3]:
#book_titles = list()
#book_stories = list()
#book_keyPicks = list()

In [5]:
driver = webdriver.Chrome(options=chrome_options)
for url in book_page_urls:
    try:
        driver.get(url)
        time.sleep(3)
        title = driver.find_element(By.CSS_SELECTOR,'#contents > div.prod_detail_header > div > div.prod_detail_title_wrap > div > div.prod_title_box.auto_overflow_wrap > div.auto_overflow_contents > div > h1 > span')
        book_titles.append(title.text)
    except:
        book_titles.append('나중에 채우기!')

    try:
        # 키워드 Pick 수집
        keywordPicks = driver.find_elements(By.CSS_SELECTOR,'div.tab_list_wrap > ul >li> a > span.tab_text')
        keywordPick = list()
        for keyword in keywordPicks:
            keywordPick.append(keyword.text)
        book_keyPicks.append(keywordPick)
    except:
        book_keyPicks.append([])

    try:
         # 책 소개 수집
        story = driver.find_element(By.CSS_SELECTOR,'div.intro_bottom')
        book_stories.append(story.text)
    except:
        book_stories.append('')
        
driver.quit()

In [4]:
len(book_page_urls)

1000

In [6]:
print(len(book_titles))
print(len(book_stories))
print(len(book_keyPicks))

1000
1000
1000


In [7]:
book_df = pd.DataFrame([book_titles,book_stories,book_keyPicks],index=['상품명','책소개','키워드']).T
def delete_null(keywordList):
    keyword_list = list()
    for x in range(len(keywordList)):
        if keywordList[x] != '':
            keyword_list.append(keywordList[x])
    return keyword_list
book_df['키워드'] = book_df['키워드'].apply(delete_null)

book_category = ['소설','시/에세이','인문','경제/경영','자기계발']
file_name = book_category[2] + '키워드 Pick.xlsx'
book_df.to_excel(file_name)

In [8]:
book_df

,상품명,책소개,키워드
0,문과 남자의 과학 공부,"과학 공부로 길어 올린 생명과 우주에 관한 진실, 인간과 사회를 이해하는 방법\n“...","[뇌과학, 양자 역학, 회한, 우주, 문과, 사유]"
1,도둑맞은 집중력,"★아마존, 월 스트리트 저널 올해의 책★\n★애덤 그랜트·수전 케인·힐러리 클린턴 ...","[주의력, 비만, 증가, 유행병, 정크푸드, 테크]"
2,모든 삶은 흐른다,“이 책은 흐르는 삶 속에서 ‘자기 자신’이라는 유일한 섬이 되는 길을 안내한다. ...,"[바다, 곡예, 이해인, 레르, 프랑스 최고, 상어]"
3,내면소통,“나와 세상을 바꾸는 힘은 불굴의 의지가 아니라\n실패를 두려워하지 않는 마음근력에...,"[뇌과학, 편도체, 경기력 향상, 뇌 발달, 블랭킷, 선문답]"
4,"2023 큰별쌤 최태성의 별별한국사 기출 500제 한국사능력검정시험 심화(1, 2,...","""2023년 65회 시험부터 2021년 56회 시험까지 최신 기출문제 10회분, 총...","[앞표지, 선택지, 킬러 문항, 열매, 난도, 리더]"
...,...,...,...
995,표준,제4판 표준 중세 국어 문법론에서는 지난 10년 동안에 나온 중세어 관련 문법 업적...,"[품사론, 형태론, 통사론, 표기법, 겹문장, 발음]"
996,삼국지연의보다 재미있는 정사 삼국지 1,20만 구독자가 열광한 화제의 유튜브 역사채널 〈써에이스쇼〉의 삼국지\n누적 조회 ...,"[판타지, 황건적, 유비, 컬러 삽화, 위촉, 실제 역사]"
997,나도 손글씨 바르게 쓰면 소원이 없겠네(핸디 워크북),아무때나 시간 날 때 틈틈이\n지하철ㆍ학교ㆍ회사ㆍ카페ㆍ집 어느 장소에서도\n편하게 ...,"[악필, 교정, 원고지, 정자체, 스프링, 지하철]"
998,한자자격시험 7급(교양한자급수시험대비),,"[필순, 짜임, 모범 답안, 회분, 기출문제, 연습 문제]"
